In [ ]:
# Basic imports
import os
import sys
import pickle
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import numpy as np

# Machine learning imports
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import (
    LinearRegression, MultiTaskLasso, MultiTaskElasticNet
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_decomposition import PLSRegression

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *

# Deep learning and machine learning specific 
import torch

from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.models import (
    GatedAdditiveTreeEnsembleConfig,
    DANetConfig,
    TabTransformerConfig,
    TabNetModelConfig,
)

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

from omegaconf import DictConfig
torch.serialization.safe_globals([DictConfig])

True
1


# Leave-One-Missing-Out (LOMO) Cross-Validation

This notebook explores predictive model performance in a leave-one-missing-out cross-validation (LOMO-CV) framework using multimodal Alzheimer’s disease biomarker data. This setting evaluates the models’ robustness when both training and testing data contain missing values.

The analysis includes Linear Regression, Lasso, ElasticNet, PLS, Random Forest, XGBoost, TabNet, TabTransformer, and DANet models across four cognitive domains: Memory, Executive Function, Visuospatial Function, and Language.

In LOMO-CV, we randomly select 20 subjects with missing data as test cases. For each, imputers are trained on the remaining training data (1,185 subjects) and applied to impute the test subject. Predictive models trained on the imputed training data are then used to predict outcomes for the test subject. Both imputation and prediction are performed without any retraining on the test data to avoid leakage. Model performance is measured using mean absolute error (MAE), mean square error (MSE) and Pearson correlation, with results summarized for each cognitive domain.

--- 

### Notebook structure
1. Leave-One-Missing-Out (LOMO-CV)
2. MRI-Only Model Evaluation
3. Results Summary and Reporting


## Load data 

In [2]:
data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
dict_select = data["dict_select"]

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

In [3]:
idx_na = list(df_X.isna().any(axis=1))
idx_nona = list(~df_X.isna().any(axis=1))

# Identify unique RIDs for rows with and without missing data
rid_na = df_all.loc[idx_na, "RID"].unique()
rid_nona = df_all.loc[idx_nona, "RID"].unique()

# Identify RIDs eligible for test set (not in train set)
eligible_rid_na = np.setdiff1d(rid_na, rid_nona)

df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("int", errors='ignore')


single_measurement_rids = (
    df_all[df_all["RID"].isin(eligible_rid_na)]
    .groupby("RID")
    .filter(lambda x: len(x) == 1)
    .RID.unique()
)

test_rids = np.random.choice(single_measurement_rids, size=20, replace=False)

# Create boolean mask for the test set
idx_test = df_all["RID"].isin(test_rids)

# Create boolean mask for the train set (all not in test)
idx_train = ~idx_test

print(idx_test.sum())

20


# Leave-One-Missing-Out (LOMO-CV)

In [4]:
random_state=42

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("KNNImputer_5", KNNImputer(n_neighbors=5)),
   # ("IterativeImputer", IterativeImputer(max_iter=100, random_state=42)),
]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("KNNImputer1", KNNImputer(n_neighbors=1)),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.1, 'l1_ratio': 0.1})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.8776807051588262, 'learning_rate': 0.13329520360246094, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.5924272277627636})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [5]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=ordinal_features
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [6]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}") 

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: LinearRegression
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskLasso
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: RandomForestRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: XGBoostRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: TabNetRegressor_custom
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: PLSRegressi

In [7]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: LinearRegression
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskLasso
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: RandomForestRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: XGBoostRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: TabNetRegressor_custom
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer1, Model: PLSRegressi

In [8]:
test_indices = [i for i, val in enumerate(idx_test) if val]

In [9]:
# Initialize HDF5 file
results_file = '../pickle/training_4_loona_dict_results.pickle'

if os.path.exists(results_file): 
    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : (df_X.shape[0]-1, df_X.shape[1]), 
            "test_shape": (1, df_X.shape[1]),
            "test_rid": df_all.iloc[test_indices]["RID"].tolist()
        }

    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want
    
    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue
    
    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X.loc[idx_train]
        df_X_test = df_X.loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )

            dict_results["imputation_time"].append(fold_dict_results["imputation_time"])  
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])    

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)
    
    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'LinearRegression']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'MultiTaskLasso']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'RandomForestRegressor']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'XGBoostRegressor']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'TabNetRegressor_default']
Skipping existing combination (subset match): ['KNNImputer1', 'KNNImputer_5', 'TabNetR

In [10]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
with open('../pickle/training_4_loona_dict_results.pickle', "rb") as input_file:
    dict_results_loo_na = pickle.load(input_file)

# MRI-Only Model Evaluation

In [12]:
idx_na = list(df_X.isna().any(axis=1))
idx_nona = list(~df_X.isna().any(axis=1))

# Identify unique RIDs for rows with and without missing data
rid_na = df_all.loc[idx_na, "RID"].unique()
rid_nona = df_all.loc[idx_nona, "RID"].unique()

# Identify RIDs eligible for test set (not in train set)
eligible_rid_na = np.setdiff1d(rid_na, rid_nona)

df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("int", errors='ignore')


single_measurement_rids = (
    df_all[df_all["RID"].isin(eligible_rid_na)]
    .groupby("RID")
    .filter(lambda x: len(x) == 1)
    .RID.unique()
)

test_rids = np.random.choice(single_measurement_rids, size=20, replace=False)

# Create boolean mask for the test set
idx_test = df_all["RID"].isin(test_rids)

# Create boolean mask for the train set (all not in test)
idx_train = ~idx_test

print(idx_test.sum())

20


In [13]:
random_state=42
n_imputation_iter = 10

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("NoImputer", KNNImputer(n_neighbors=1)),

]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("NoImputer", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.1, 'l1_ratio': 0.1})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.8776807051588262, 'learning_rate': 0.13329520360246094, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.5924272277627636})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [14]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y.columns.tolist(),
    continuous_cols=dict_select["MRIth"],
    categorical_cols=[]
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=True,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__
predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]


In [15]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: LinearRegression
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: RandomForestRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_default
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_custom
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: PLSRegression_4_components
Continuous Imputer: NoImputer, Ordinal 

In [16]:
test_indices = [i for i, val in enumerate(idx_test) if val]

In [17]:
# Initialize HDF5 file
results_file = '../pickle/training_4_loona_dict_results.pickle'

if os.path.exists(results_file): 
    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

In [18]:
test_rids = all_dict_results[0]["params"]["test_rid"]

# Create boolean mask for the test set
idx_test = df_all["RID"].isin(test_rids)

# Create boolean mask for the train set (all not in test)
idx_train = ~idx_test

print(idx_test.sum())

test_indices = [i for i, val in enumerate(idx_test) if val]

20


In [19]:
all([results["params"]["test_rid"] == test_rids for results in all_dict_results])

False

In [20]:
len(all_dict_results)

60

In [21]:
test_indices

[480,
 501,
 569,
 632,
 733,
 899,
 952,
 1113,
 1145,
 1219,
 1781,
 1848,
 1961,
 2104,
 2150,
 2209,
 2325,
 2382,
 2420,
 2833]

In [22]:
# Initialize HDF5 file
results_file = '../pickle/training_4_loona_dict_results.pickle'

if os.path.exists(results_file): 
    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

In [23]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : (df_X.shape[0]-1, df_X.shape[1]), 
            "test_shape": (1, df_X.shape[1]),
            "test_rid": df_all.iloc[test_indices]["RID"].tolist()
        }

    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want
    
    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue
    
    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
        df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )

            dict_results["imputation_time"].append(fold_dict_results["imputation_time"])  
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])    

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)
    
    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'LinearRegression']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskLasso']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'RandomForestRegressor']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'XGBoostRegressor']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'TabNetRegressor_default']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'TabNetRegressor_custom']
Skipping existing combination (s

In [24]:
len(all_dict_results)

60

# Results Summary and Reporting

In [25]:
with open('../pickle/training_4_loona_dict_results.pickle', "rb") as input_file:
    dict_results_loona = pickle.load(input_file)

In [26]:
def generate_metric_table(
    results_list,
    targets,
    metric_name="r2",
    source="Adjusted",
    csv_filename=None,
    sort_order="descending"
):
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, explained_variance_score
    from scipy.stats import pearsonr
    import pandas as pd
    import numpy as np

    # Choose the key
    key = "results_adj" if source.lower() == "adjusted" else "results_org"
    df_rows = []

    for res in results_list:
        result_blocks = res.get(key, [])
        if not isinstance(result_blocks, list) or len(result_blocks) == 0:
            continue

        # Aggregate all predictions across folds
        y_preds_all = {target: [] for target in targets}
        y_tests_all = {target: [] for target in targets}

        for fold in result_blocks:
            y_pred = fold["y_pred"]
            y_test = fold["y_test"]

            if y_pred.shape != y_test.shape:
                continue

            for i, target in enumerate(targets):
                if y_pred.shape[1] <= i:
                    continue
                y_preds_all[target].append(y_pred[0, i])
                y_tests_all[target].append(y_test[0, i])

        # Compute metric for each target
        target_metrics = []
        for target in targets:
            y_preds = y_preds_all[target]
            y_tests = y_tests_all[target]

            if len(y_preds) < 2:
                metric = float("nan")
            else:
                if metric_name == "r2":
                    metric = r2_score(y_tests, y_preds)
                elif metric_name == "mae":
                    metric = mean_absolute_error(y_tests, y_preds)
                elif metric_name == "mse":
                    metric = mean_squared_error(y_tests, y_preds)
                elif metric_name == "explained_variance":
                    metric = explained_variance_score(y_tests, y_preds)
                elif metric_name == "corr":
                    metric = pearsonr(y_tests, y_preds)[0]
                else:
                    raise ValueError(f"Unsupported metric: {metric_name}")

            target_metrics.append(metric)

        # Compute mean ± std across targets
        values = np.array(target_metrics, dtype=np.float64)
        mean_val = np.nanmean(values)
        std_val = np.nanstd(values)

        # Get times
        imp_times = np.array(res.get("imputation_time", []), dtype=np.float64)
        fit_times = np.array(res.get("fitting_time", []), dtype=np.float64)

        row = {
            "continuous_imputer": res["params"].get("continuous_imputer"),
            "ordinal_imputer": res["params"].get("ordinal_imputer"),
            "model": res["params"].get("model"),
            **{target: val for target, val in zip(targets, target_metrics)},
            "mean ± std": f"{mean_val:.3f} ± {std_val:.3f}",
            "imp_time (s)": f"{np.nanmean(imp_times):.1f} ± {np.nanstd(imp_times):.1f}",
            "fit_time (s)": f"{np.nanmean(fit_times):.1f} ± {np.nanstd(fit_times):.1f}",
            "_sort_val": mean_val  # hidden sort key
        }
        df_rows.append(row)

    df = pd.DataFrame(df_rows)

    # Sort by mean value
    df = df.sort_values(by="_sort_val", ascending=(sort_order == "ascending")).drop(columns="_sort_val")

    # Optional: Save to CSV
    if csv_filename:
        df.to_csv(csv_filename, index=False)

    # Format for LaTeX output
    latex_df = df.copy()
    for target in targets:
        latex_df[target] = latex_df[target].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "–")

    latex_output = latex_df.to_latex(index=False, escape=False)

    return df, latex_output


In [27]:
latex_df, latex_corr = generate_metric_table(
    results_list=dict_results_loona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='corr',
    source="Adjusted",
    csv_filename="../tables/4_training_loona_corr_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_corr)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer & XGBoostRegressor & 0.759 & 0.858 & 0.808 & 0.877 & 0.825 ± 0.046 & 2.5 ± 0.0 & 1.0 ± 0.0 \\
KNNImputer_5 & KNNImputer & RandomForestRegressor & 0.726 & 0.822 & 0.784 & 0.829 & 0.790 ± 0.041 & 2.5 ± 0.1 & 39.4 ± 0.3 \\
KNNImputer_5 & KNNImputer & PLSRegression_4_components & 0.825 & 0.759 & 0.754 & 0.812 & 0.787 ± 0.032 & 2.6 ± 0.0 & 0.1 ± 0.0 \\
KNNImputer_5 & KNNImputer & MultiTaskElasticNet & 0.804 & 0.752 & 0.708 & 0.836 & 0.775 ± 0.049 & 2.7 ± 0.1 & 0.0 ± 0.0 \\
KNNImputer_5 & KNNImputer & TabNetRegressor_default & 0.800 & 0.790 & 0.738 & 0.766 & 0.774 ± 0.024 & 2.6 ± 0.0 & 12.6 ± 0.7 \\
KNNImputer_5 & KNNImputer & MultiTaskElasticNet_tuned & 0.816 & 0.697 & 0.756 & 0.797 & 0.766 ± 0.045 & 2.7 ± 0.1 & 1.4 ± 0.1 \\
KNNImputer_5 & KNNImputer & GatedAdditiveTreeEnsembleConfig_tab &

In [28]:
latex_df, latex_r2 = generate_metric_table(
    results_list=dict_results_loona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='r2',
    source="Adjusted",
    csv_filename="../tables/4_training_loona_r2_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_r2)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer & XGBoostRegressor & 0.521 & 0.675 & 0.476 & 0.646 & 0.580 ± 0.083 & 2.5 ± 0.0 & 1.0 ± 0.0 \\
KNNImputer_5 & KNNImputer & RandomForestRegressor & 0.480 & 0.627 & 0.496 & 0.570 & 0.543 ± 0.059 & 2.5 ± 0.1 & 39.4 ± 0.3 \\
KNNImputer_5 & KNNImputer & TabNetRegressor_default & 0.590 & 0.528 & 0.482 & 0.480 & 0.520 ± 0.045 & 2.6 ± 0.0 & 12.6 ± 0.7 \\
KNNImputer_5 & KNNImputer & MultiTaskElasticNet_tuned & 0.594 & 0.404 & 0.450 & 0.466 & 0.478 ± 0.071 & 2.7 ± 0.1 & 1.4 ± 0.1 \\
KNNImputer_5 & KNNImputer & XGBoostRegressor_tuned & 0.499 & 0.542 & 0.313 & 0.560 & 0.478 ± 0.098 & 2.5 ± 0.1 & 4.6 ± 0.1 \\
KNNImputer_5 & KNNImputer & PLSRegression_4_components & 0.606 & 0.445 & 0.400 & 0.444 & 0.474 ± 0.078 & 2.6 ± 0.0 & 0.1 ± 0.0 \\
KNNImputer_5 & KNNImputer & MultiTaskLasso_tuned & 0.582 & 0.3

In [29]:
latex_df, latex_mae = generate_metric_table(
    results_list=dict_results_loona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mae',
    source="Adjusted",
    csv_filename="../tables/4_training_loona_mae_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer1 & GatedAdditiveTreeEnsembleConfig_tab & 0.519 & 0.547 & 0.522 & 0.513 & 0.525 ± 0.013 & 4.2 ± 0.3 & 36.4 ± 1.0 \\
KNNImputer_5 & KNNImputer1 & PLSRegression_4_components & 0.496 & 0.544 & 0.511 & 0.568 & 0.530 ± 0.028 & 4.1 ± 0.4 & 0.7 ± 0.1 \\
KNNImputer_5 & KNNImputer1 & MultiTaskElasticNet_tuned & 0.457 & 0.569 & 0.568 & 0.560 & 0.539 ± 0.047 & 4.0 ± 0.5 & 2.7 ± 0.3 \\
KNNImputer_5 & KNNImputer1 & XGBoostRegressor_tuned & 0.501 & 0.561 & 0.571 & 0.555 & 0.547 ± 0.027 & 4.1 ± 0.6 & 408.4 ± 2.1 \\
KNNImputer_5 & KNNImputer1 & XGBoostRegressor & 0.529 & 0.554 & 0.516 & 0.610 & 0.553 ± 0.036 & 4.1 ± 0.6 & 165.8 ± 2.6 \\
KNNImputer_5 & KNNImputer1 & MultiTaskLasso_tuned & 0.467 & 0.613 & 0.604 & 0.606 & 0.573 ± 0.061 & 4.0 ± 0.6 & 1.6 ± 0.3 \\
KNNImputer_5 & KNNImputer1 & LinearRegress

In [30]:
latex_df, latex_mse = generate_metric_table(
    results_list=dict_results_loona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mse',
    source="Adjusted",
    csv_filename="../tables/4_training_loona_mse_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mse)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer1 & PLSRegression_4_components & 0.388 & 0.420 & 0.337 & 0.537 & 0.420 ± 0.074 & 4.1 ± 0.4 & 0.7 ± 0.1 \\
KNNImputer_5 & KNNImputer1 & GatedAdditiveTreeEnsembleConfig_tab & 0.389 & 0.443 & 0.374 & 0.515 & 0.430 ± 0.055 & 4.2 ± 0.3 & 36.4 ± 1.0 \\
KNNImputer_5 & KNNImputer1 & MultiTaskElasticNet_tuned & 0.321 & 0.480 & 0.421 & 0.577 & 0.450 ± 0.093 & 4.0 ± 0.5 & 2.7 ± 0.3 \\
KNNImputer_5 & KNNImputer1 & XGBoostRegressor & 0.435 & 0.489 & 0.367 & 0.647 & 0.484 ± 0.103 & 4.1 ± 0.6 & 165.8 ± 2.6 \\
KNNImputer_5 & KNNImputer1 & XGBoostRegressor_tuned & 0.396 & 0.543 & 0.454 & 0.586 & 0.495 ± 0.074 & 4.1 ± 0.6 & 408.4 ± 2.1 \\
KNNImputer_5 & KNNImputer1 & MultiTaskLasso_tuned & 0.354 & 0.562 & 0.528 & 0.653 & 0.524 ± 0.109 & 4.0 ± 0.6 & 1.6 ± 0.3 \\
KNNImputer_5 & KNNImputer1 & LinearRegress